In [1]:
from customdataset import CustomDataset
from SSD import *
from train_step import train_step
from eval_step import test_step
import os
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [2]:
PATH=os.getenv("HOME")+"/aiffel/ECG_data/physionet.org/files/circor-heart-sound/1.0.3/training_data"
file_list = os.listdir(PATH)
txt_list = [os.path.join(PATH, file) for file in file_list if file.endswith(".txt")]

# 환자 아이디를 훈련 데이터셋과 테스트 데이터셋으로 나눔
train_patient_txt, test_patient_txt = train_test_split(txt_list, test_size=0.9, random_state=42)

# 결과 출력
#print("Train Patient IDs:", train_patient_txt[:1])
#print("Test Patient IDs:", test_patient_txt[:1])

#path, txt_list, filter_params, target_size, th, resizing
dataset = CustomDataset(PATH, train_patient_txt, target_size=(300, 300), th=5, resizing=True)


def my_collate_fn(batch):
    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])  # sample[0]은 화상 gt
        targets.append(torch.FloatTensor(sample[1]))  # sample[1]은 어노테이션 gt

    imgs = torch.stack(imgs, dim=0)
    return imgs, targets
BATCHSIZE = 8
train_dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=True, collate_fn=my_collate_fn)

# train_step

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model("Train", input_channels=1, is_freeze=False)
#model_weight_path='./objectdetection_model/ssd300_weight_100.pth'
#weight = torch.load(model_weight_path)
#model.load_state_dict(weight)

In [ ]:
_optim_type = "Adam"
_lr=2e-3
_model_name = "MnetSSD"
_is_freeze = "False"

train_step(model, train_dataloader,epoch_num = 100, batchsize=BATCHSIZE,
           optim_type = _optim_type, lr=_lr, device = DEVICE, model_name = _model_name,
           is_wandb=False, is_freeze=_is_freeze)


# test_step

In [3]:
_optim_type = "Adam"
_lr=2e-3
_model_name = "MnetSSD"
_is_freeze = "False"


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = build_model("Test", input_channels=1)
model_weight_path='./objectdetection_model/'+f'{_model_name}_weight_100_{BATCHSIZE}_{_optim_type}_{_is_freeze}.pth'
weight = torch.load(model_weight_path)
model.load_state_dict(weight)
model.eval()

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
recall, precison, _ = test_step(model, train_dataloader, device = DEVICE)

print(total_TP_1,total_TP_2)
print(total_FP,total_FN)

In [5]:
print(recall, precison)

0.9571545419764123 0.9616425424291875


In [7]:
!pip freeze | grep -E "torch" >> requirements.txt
!pip freeze | grep -E "skimage" >> requirements.txt
!pip freeze | grep -E "numpy" >> requirements.txt
!pip freeze | grep -E "librosa" >> requirements.txt
!pip freeze | grep -E "wandb" >> requirements.txt
!pip freeze | grep -E "scipy" >> requirements.txt
!pip freeze | grep -E "time" >> requirements.txt
!pip freeze | grep -E "pandas" >> requirements.txt
!pip freeze | grep -E "matplotlib" >> requirements.txt

In [7]:
!pip freeze | grep -E "numpy" >> requirements.txt